In [184]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
from urllib.parse import quote

In [165]:
driver = webdriver.Chrome('chromedriver')


## 1. Instagram 에 접속하여 로그인¶

In [168]:
insta_url='http://www.instaram.com'
driver.get(insta_url)
time.sleep(1)


In [169]:
with open('password.txt') as f:
    password = f.read(12)


In [170]:
email = 'raient007'
input_email = driver.find_element_by_css_selector('._2hvTZ.pexuQ.zyHYP')
input_email.clear()
input_email.send_keys(email)

In [171]:
input_pwd = driver.find_element_by_name('password')
input_pwd.clear()
input_pwd.send_keys(password)
input_pwd.submit()
time.sleep(1)

In [172]:
#  로그인 정보 저장 -나중에 하기
try:
    driver.find_element_by_css_selector('.sqdOP.yWX7d.y3zKF').click()
    time.sleep(1)
except:
    pass

In [173]:
#  알림설정 -나중에 하기
try:
    driver.find_element_by_css_selector('.aOOlW.HoLwm').click()
    time.sleep(1)
except:
    pass

## 2. 제주도맛집 으로 검색


In [175]:

from urllib.parse import quote
keyword = '제주도맛집'
search_url = 'https://www.instagram.com/explore/tags/'
url = f'{search_url}{quote(keyword)}'
url

'https://www.instagram.com/explore/tags/%EC%A0%9C%EC%A3%BC%EB%8F%84%EB%A7%9B%EC%A7%91'

In [176]:
driver.get(url)
time.sleep(5)

In [177]:
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(1)

## 4. 게시글 정보 가져오기

In [192]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [193]:
import unicodedata
try:
    content =soup.select_one('div.C4VMK >span').text
    content =unicodedata.normalize('NFC', content)
except:
    content = ' '    
content


'오늘은 재택근무를 하는 날이라,, 얼루기랑 집근처 식당에서 외식! 30분 만에 호다닥 먹었지만 기분 전환도 되고 참 좋았당~ 츄베릅!😋'

In [100]:
# 본문 내용에서 해쉬태그 가져오기 , ' #[^\s#,\\]+'

In [194]:
import re
tags = re.findall(r'#[^\s#,\\]+', content)
tags

[]

In [195]:
#  3) 작성일자 정보 가져오기
#date = soup.select_one('time.FH9sR.Nzb55')['datetime'][:10]
#date
date = soup.select_one('time.FH9sR.Nzb55')['datetime'][:10]
date

'2021-08-19'

In [196]:
# 4) 좋아요 수 가져오기
try:
    like = soup.select_one('div.Nm9Fw').text[4:-1]
except:
    like = 0
like

'8'

In [197]:
# 5) 위치정보 가져오기
try:    
    place = soup.select('div.M30cS')[0].text
    place = unicodedata.normalize('NFC', place)
except:
    place = ''
place

'커뮤니테이블'

In [81]:
# 5. 다음 게시글로 이동

In [191]:
driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
time.sleep(2)

In [198]:
# 6을 하기 위해서 팝업 화면 닫기, x 클릭하기
driver.find_element_by_xpath('/html/body/div[6]/div[3]/button').click()


In [199]:
#6 여러개의 게시글 가져오기

from tqdm.notebook import tqdm
'''
def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    try:
        content =soup.select_one('div.C4VMK >span').text
        content =unicodedata.normalize('NFC', content)
    except:
        content = ' '    
    
    tags = re.findall(r'#[^\s#,\\]+', content)
    try:
        date = soup.select_one('time.FH9sR.Nzb55')['datetime'][:10]
    except:
        date =''    
    
    try:
        like = soup.select_one('div.Nm9Fw').text[4:-1]
    except:
        like = 0
    
    try:    
        place = soup.select('div.M30cS')[0].text
        place = unicodedata.normalize('NFC', place)
    except:
        place = ''

    return [content, tags, date, like, place]
'''

"\ndef get_content(driver):\n    html = driver.page_source\n    soup = BeautifulSoup(html, 'html.parser')\n\n    try:\n        content =soup.select_one('div.C4VMK >span').text\n        content =unicodedata.normalize('NFC', content)\n    except:\n        content = ' '    \n    \n    tags = re.findall(r'#[^\\s#,\\]+', content)\n    try:\n        date = soup.select_one('time.FH9sR.Nzb55')['datetime'][:10]\n    except:\n        date =''    \n    \n    try:\n        like = soup.select_one('div.Nm9Fw').text[4:-1]\n    except:\n        like = 0\n    \n    try:    \n        place = soup.select('div.M30cS')[0].text\n        place = unicodedata.normalize('NFC', place)\n    except:\n        place = ''\n\n    return [content, tags, date, like, place]\n"

In [210]:
def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    try:
        content = soup.select_one('div.C4VMK > span').text
        content = unicodedata.normalize('NFC', content)
    except:
        content = ' '
    
    date = soup.select_one('time.FH9sR.Nzb55')['datetime'][:10]
    tags = re.findall(r'#[^\s#,\\]+', content)
    

    try:
        like = soup.select_one('div.Nm9Fw').text[4:-1]
    except:
        like = 0
    try:
        place = soup.select('div.M30cS')[0].text
        place = unicodedata.normalize('NFC', place)
    except:
        place = ''

    return [content, date, like, place, tags]

In [211]:
keyword = '제주도맛집'
search_url = 'https://www.instagram.com/explore/tags/'
url = f'{search_url}{quote(keyword)}'
driver.get(url)
time.sleep(5)
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(1)

In [212]:
# 김재용
results = []
for i in tqdm(range(1)):
    try:
        row = get_content(driver)
        results.append(row)
        driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
        time.sleep(2)
    except:
        time.sleep(3)
        driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()


  0%|          | 0/1 [00:00<?, ?it/s]

In [220]:
# 교수님
results = []
for _ in tqdm(range(50)):
    try:
        row = get_content(driver)
        results.append(row)
        driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
        time.sleep(2)
    except:
        time.sleep(3)
        driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
        time.sleep(2)

  0%|          | 0/50 [00:00<?, ?it/s]

In [221]:
df = pd.DataFrame(results, columns=['content','date','like','place','tags'])
df.head()

,content,date,like,place,tags
0,"협찬제주애월맛집 ""랍스터빈""주소 : 제주시 애월읍 애월해안로 752전화 : 064-...",2021-08-18,0,,"[#애월맛집, #애월카페, #애월읍맛집, #제주애월맛집, #제주도맛집, #제주맛집,..."
1,<광고>#우도 들어가기전에 꼭 #산도롱맨도롱#갈비국수 꼭 드셔보세요.✔손소독제 항시...,2021-08-18,0,,"[#우도, #산도롱맨도롱, #갈비국수, #제주맛집, #제주도맛집, #성산맛집, #성..."
2,오늘도 맛있고 두툼하고 부드럽게 준비 잘 해두었어요🙋‍♀️곧 시즌마감 예정인 애플망...,2021-08-19,842,월정리해변,[#안녕하샌]
3,그 유명한 #연돈 에 다녀왔습니다. 그런데ㅎ;;;⠀⠀⠀⠀⠀⠀⠀어마무시한 웨이팅으로 ...,2021-08-18,"4,626",연돈,"[#연돈, #제주도연돈, #테이블링, #등심까스, #수제볼카츠, #아토짱_제주, #..."
4,제주도의 핫이슈!! 예약문의 폭주!! 애월 한라봉스시 스폐셜 메뉴 천국의계단(2-3...,2021-08-18,290,제주도 애월읍,"[#애월맛집, #제주도맛집, #제주맛집, #애월스시, #제주스시, #애월횟집, #제..."


In [222]:
df.columns


Index(['content', 'date', 'like', 'place', 'tags'], dtype='object')

In [223]:
driver.close()

In [224]:
jeju_df = pd.DataFrame([])

In [228]:
for keyword in ['제주도맛집', '제주맛집', '제주도관광', '제주여행']:
    df = pd.read_csv(f'data/{keyword}.csv')
    jeju_df = jeju_df.append(df)

In [231]:
jeju_df.tail()


,content,date,like,place,tags
109,"ო̤̜어깨 깡패 😏8, 9월 예약 받고 있습니다 🤗문의는 프로필의 오픈 카톡으로 부...",[],2021-08-19,19,제주시청 근처
110,.막 찍어도 화보 같은 월정리 카페 콧수염고객님의 멋진 사진 감사합니다 🤳제주시 구...,"['#구좌읍카페', '#함덕맛집', '#제주', '#제주도맛집', '#월정리', '...",2021-08-19,3,NaN
111,사장님 알로뷰 …❤️,[],2021-08-19,43,One and only - 원앤온리 제주
112,#제주여행 #1일차#사진못찍음,"['#제주여행', '#1일차', '#사진못찍음']",2021-08-19,4,NaN
113,"부랴부랴 택배 싸놓고,오후는 율이와 피크닉 🧺 수업준비와 주문제작건은 야근 당첨😭제...","['#연이네다락방', '#엄만야근해야해', '#제주피크닉', '#풍경이다했다', '...",2021-08-19,9,선흘리
